# HR Analytics

# Part 1 - Define

The purpose of this project is to understand the factors that cause employees to leave their jobs.  Understanding this question is critical because companies spend significant amounts of time and money hiring new employees and training them to the needed level. 

In [1]:
# Importing libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


# Part 2 - Discover

### Loading the data

In [5]:
df = pd.read_csv('HR-data.csv.txt', delimiter = ',')
df.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary
0,0.38,0.53,2,157,3,0,1,0,sales,low
1,0.80,0.86,5,262,6,0,1,0,sales,medium
2,0.11,0.88,7,272,4,0,1,0,sales,medium
3,0.72,0.87,5,223,5,0,1,0,sales,low
4,0.37,0.52,2,159,3,0,1,0,sales,low


In [9]:
df.describe(include = ['O'])

,sales,salary
count,14999,14999
unique,10,3
top,sales,low
freq,4140,7316
